In [15]:
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV


In [16]:
#Limpieza del dataset de test

oportunidades = pd.read_csv("Train_TP2_Datos_2020-2C.csv")
oportunidades['Account_Created_Date'] = pd.to_datetime(oportunidades['Account_Created_Date'])
oportunidades['Opportunity_Created_Date'] = pd.to_datetime(oportunidades['Opportunity_Created_Date'])
oportunidades['Quote_Expiry_Date'] = pd.to_datetime(oportunidades['Quote_Expiry_Date'])
oportunidades['Last_Modified_Date'] = pd.to_datetime(oportunidades['Last_Modified_Date'])
oportunidades['Planned_Delivery_Start_Date'] = pd.to_datetime(oportunidades['Planned_Delivery_Start_Date'])
oportunidades['Planned_Delivery_End_Date'] = pd.to_datetime(oportunidades['Planned_Delivery_End_Date'])
oportunidades_japon = (oportunidades.loc[oportunidades['Region'] == 'Japan'])
oportunidades = (oportunidades.loc[oportunidades['Region'] != 'Japan'])
#oportunidades_japon = oportunidades_japon.iloc[:,:3]
oportunidades_japon['Territory'] = oportunidades_japon['Territory'].replace({'None':'Japan'})
oportunidades = pd.concat([oportunidades, oportunidades_japon], axis=0)
oportunidades['Region'] = oportunidades['Region'].replace({'Japan':'APAC', 'Middle East':'EMEA'})
oportunidades[oportunidades.select_dtypes(['object']).columns] = oportunidades.select_dtypes(['object']).apply(lambda x: x.astype('category'))
#oportunidades = oportunidades.loc[oportunidades['Territory'] != 'None']
#oportunidades = oportunidades.dropna()

filtro_terminos_entrega = oportunidades['Pricing, Delivery_Terms_Quote_Appr']\
     == oportunidades['Pricing, Delivery_Terms_Approved']

filtro_codigo_burocratico = oportunidades['Bureaucratic_Code_0_Approval']\
     == oportunidades['Bureaucratic_Code_0_Approved']

oportunidades = oportunidades.drop(['Pricing, Delivery_Terms_Quote_Appr'\
    ,'Pricing, Delivery_Terms_Approved', 'Bureaucratic_Code_0_Approval'\
        , 'Bureaucratic_Code_0_Approved', 'Submitted_for_Approval'], axis = 'columns')

oportunidades["Moneda_Convertida"] = (oportunidades['ASP'] / oportunidades['ASP_(converted)'])
oportunidades_posibles = (filtro_terminos_entrega & filtro_codigo_burocratico)

oportunidades.insert(3,'Es_Oportunidad_Posible', oportunidades_posibles)

oportunidades['Es_Oportunidad_Posible'] = oportunidades['Es_Oportunidad_Posible'].replace(
    {True:1, False:0})

oportunidades

,ID,Region,Territory,Es_Oportunidad_Posible,Bureaucratic_Code,Account_Created_Date,Source,Billing_Country,Account_Name,Opportunity_Name,...,Delivery_Year,Actual_Delivery_Date,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Stage,Prod_Category_A,Moneda_Convertida
0,27761,EMEA,None,1,Bureaucratic_Code_4,2015-06-16,None,Netherlands,Account_Name_619,Opportunity_Name_12598,...,2016,NaT,10,EUR,5272800.0,EUR,5272800.0,Closed Lost,Prod_Category_A_None,0.884098
1,27760,EMEA,None,1,Bureaucratic_Code_4,2015-06-16,None,Netherlands,Account_Name_619,Opportunity_Name_12600,...,2016,NaT,0,EUR,48230.0,EUR,48230.0,Closed Won,Prod_Category_A_None,0.884100
2,27446,Americas,NW America,1,Bureaucratic_Code_4,2015-04-21,Source_7,United States,Account_Name_1794,Opportunity_Name_469,...,2016,NaT,0,USD,83865.6,USD,83865.6,Closed Won,Prod_Category_A_None,1.000000
3,16808,Americas,NW America,0,Bureaucratic_Code_5,2013-07-27,Source_11,United States,Account_Name_1201,Opportunity_Name_415,...,2018,NaT,14,USD,7421881.5,USD,7421881.5,Closed Lost,Prod_Category_A_None,1.000000
4,16805,Americas,NW America,0,Bureaucratic_Code_5,2013-07-27,Source_11,United States,Account_Name_1201,Opportunity_Name_851,...,2018,NaT,25,USD,13357192.5,USD,13357192.5,Closed Lost,Prod_Category_A_None,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16922,18519,APAC,Japan,1,Bureaucratic_Code_4,2013-07-28,Source_7,Singapore,Account_Name_1294,Opportunity_Name_8874,...,2016,NaT,0,USD,13200.0,USD,6422400.6,Closed Won,Prod_Category_A_None,1.000000
16923,27689,APAC,Japan,1,Bureaucratic_Code_4,2015-06-11,Source_3,Japan,Account_Name_1074,Opportunity_Name_6488,...,2016,NaT,0,JPY,12194000.0,JPY,12194000.0,Closed Won,Prod_Category_A_None,111.229165
16924,27688,APAC,Japan,1,Bureaucratic_Code_4,2015-06-11,Source_3,Japan,Account_Name_1074,Opportunity_Name_6489,...,2016,NaT,0,JPY,11959500.0,JPY,11959500.0,Closed Won,Prod_Category_A_None,111.229165
16925,19062,APAC,Japan,1,Bureaucratic_Code_4,2013-08-11,Source_11,Japan,Account_Name_1522,Opportunity_Name_869,...,2016,NaT,0,JPY,17537000.0,JPY,17537000.0,Closed Won,Prod_Category_A_None,111.229477


In [17]:
#Pasamos Los Features del otro notebook
#Encoding para el dataset de testeo, con los mismos que en training set.

encoding_owners = oportunidades.groupby('Opportunity_Owner').agg({'Total_Amount':'mean'})
encoding_owners.columns = ['Encoding_Vendedor']
encoding_owners = encoding_owners.reset_index()
oportunidades = oportunidades.merge(encoding_owners, how='inner', on='Opportunity_Owner')

encoding_territory = oportunidades.groupby('Territory').agg({'Total_Amount' : 'mean'})
encoding_territory.columns = ['Encoding_Territorio']
encoding_territory = encoding_territory.reset_index()
oportunidades = oportunidades.merge(encoding_territory, how='inner', on='Territory')

encoding_region = oportunidades.groupby('Region').agg({'Total_Amount' : 'mean'})
encoding_region.columns = ['Encoding_Region']
encoding_region = encoding_region.reset_index()
oportunidades = oportunidades.merge(encoding_region, how='inner', on='Region')

encoding_moneda = oportunidades.groupby('Total_Amount_Currency').agg({'Moneda_Convertida' : 'mean'})
encoding_moneda.columns = ['Encoding_Moneda']
encoding_moneda = encoding_moneda.reset_index()
oportunidades = oportunidades.merge(encoding_moneda, how='inner', on='Total_Amount_Currency')
oportunidades

oportunidades = pd.concat([oportunidades, pd.get_dummies(oportunidades['Bureaucratic_Code'])],axis='columns')

encoding_bill_c = oportunidades.groupby('Billing_Country').agg({'Total_Amount' : 'mean'})
encoding_bill_c.columns = ['Encoding_Billing_Country']
encoding_bill_c = encoding_bill_c.reset_index()
oportunidades = oportunidades.merge(encoding_bill_c, how='inner', on='Billing_Country')
oportunidades

oportunidades = pd.concat([oportunidades, pd.get_dummies(oportunidades['Account_Type'])],axis='columns')

oportunidades = pd.concat([oportunidades,pd.get_dummies(oportunidades['Delivery_Terms'])],axis='columns')

encoding_product_family = oportunidades.groupby('Product_Family').agg({'Total_Amount' : 'mean'})
encoding_product_family.columns = ['Encoding_Prod_Family']
encoding_product_family = encoding_product_family.reset_index()
oportunidades = oportunidades.merge(encoding_product_family, how='inner', on='Product_Family')

oportunidades['Opportunity_Created_Year'] = oportunidades['Opportunity_Created_Date'].dt.year
oportunidades['Opportunity_Created_Month'] = oportunidades['Opportunity_Created_Date'].dt.month
oportunidades['Año_Por_Mes'] = oportunidades['Opportunity_Created_Year'] * oportunidades['Opportunity_Created_Month']


#y_train = oportunidades['Stage']
oportunidades = oportunidades.loc[(oportunidades['Stage'] == 'Closed Won') | (oportunidades['Stage'] == 'Closed Lost')]
oportunidades['Stage'] = oportunidades['Stage'].replace({'Closed Won': 1, 'Closed Lost': 0})
y_train = oportunidades.groupby('Opportunity_ID').agg({'Stage' : 'max'})
# std_deviation_amounts = oportunidades.groupby('Opportunity_ID').agg({'TRF':'std', 'Opportunity_Created_Month':'std'})
# std_deviation_amounts.columns = ['TRF_Std', 'Opp_Created_Month_Std']
# std_deviation_amounts = std_deviation_amounts.reset_index()

#Solo usamos Bureau_Code_ 0,1,2,4,5 porque son los que tiene el set de test.
#Hacemos el groupby

# std_deviation_amounts = oportunidades.groupby('Opportunity_ID').agg({'TRF':'std'})
# std_deviation_amounts.columns = ['TRF_Std']
# std_deviation_amounts = std_deviation_amounts.reset_index()
x_train = oportunidades.groupby('Opportunity_ID').agg({'Año_Por_Mes' : 'mean',\
                                                      'Encoding_Moneda':'mean', 'Total_Amount': 'sum',\
                                                      'Total_Taxable_Amount':'mean', 'Encoding_Region':'mean',\
                                                      'Encoding_Territorio':'mean','Encoding_Vendedor':'mean',\
                                                      'Es_Oportunidad_Posible':'mean', 'Bureaucratic_Code_0':'mean',\
                                                     'Bureaucratic_Code_1':'mean','Bureaucratic_Code_2':'mean','Bureaucratic_Code_4':'mean',\
                                                     'Bureaucratic_Code_5':'mean', 'Encoding_Billing_Country':'mean',\
                                                      'Delivery_Terms_0':'mean','Delivery_Terms_1':'mean','Delivery_Terms_2':'mean',\
                                                      'Delivery_Terms_3':'mean','Delivery_Terms_4':'mean','Delivery_Terms_5':'mean',\
                                                      'Delivery_Terms_6':'mean','Delivery_Terms_7':'mean','Delivery_Terms_8':'mean',
                                                     'Encoding_Prod_Family':'mean','Account_Type_0':'mean','Account_Type_1':'mean',\
                                                      'Account_Type_2':'mean','Account_Type_4':'mean', 'Account_Type_5':'mean'})
x_train = x_train.reset_index()
#x_test = x_test.merge(std_deviation_amounts, on='Opportunity_ID', how='inner')
x_train = x_train.set_index('Opportunity_ID')
#x_test['Total_Amount_Std'] = x_test['Total_Amount_Std'].replace({np.nan:0})
#x_test['Total_Tax_Amount_Std'] = x_test['Total_Tax_Amount_Std'].replace({np.nan:0})
x_train['Total_Amount'] = (x_train['Total_Amount'] / x_train['Encoding_Moneda']) 
x_train['Total_Taxable_Amount'] = (x_train['Total_Taxable_Amount'] / x_train['Encoding_Moneda']) 
x_train['Diferencia_Amounts'] = (x_train['Total_Amount'] - x_train['Total_Taxable_Amount'])
x_train = x_train.drop(['Encoding_Moneda'], axis='columns')
#x_test['TRF_Std'] = x_test['TRF_Std'].replace({np.nan:0})
x_train

,Año_Por_Mes,Total_Amount,Total_Taxable_Amount,Encoding_Region,Encoding_Territorio,Encoding_Vendedor,Es_Oportunidad_Posible,Bureaucratic_Code_0,Bureaucratic_Code_1,Bureaucratic_Code_2,...,Delivery_Terms_6,Delivery_Terms_7,Delivery_Terms_8,Encoding_Prod_Family,Account_Type_0,Account_Type_1,Account_Type_2,Account_Type_4,Account_Type_5,Diferencia_Amounts
Opportunity_ID,,,,,,,,,,,,,,,,,,,,,
0,24180,5.963857e+06,5.963857e+06,9.546745e+05,2.096019e+06,7.674847e+05,1,0,0,0,...,0,0,0,2.962691e+06,0,0,1,0,0,0.00
1,24180,5.455106e+04,5.455106e+04,9.546745e+05,2.096019e+06,6.803130e+05,1,0,0,0,...,0,0,0,2.962691e+06,0,0,1,0,0,0.00
2,24180,8.386560e+04,8.386560e+04,1.904546e+06,1.397838e+06,2.088155e+06,1,0,0,0,...,0,0,0,6.833342e+04,0,0,0,0,1,0.00
3,24180,7.421882e+06,7.421882e+06,1.904546e+06,1.397838e+06,2.088155e+06,0,0,0,0,...,0,0,0,5.688797e+06,0,0,0,0,1,0.00
4,24180,1.335719e+07,1.335719e+07,1.904546e+06,1.397838e+06,2.088155e+06,0,0,0,0,...,0,0,0,5.688797e+06,0,0,0,0,1,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12799,24180,4.543471e+05,4.543471e+05,9.546745e+05,3.385476e+05,9.032086e+05,1,0,0,0,...,0,0,0,2.962691e+06,1,0,0,0,0,0.00
12800,24180,2.133250e+07,2.133250e+07,1.904546e+06,2.160400e+06,9.032086e+05,0,0,0,0,...,0,0,0,5.115740e+06,0,0,0,0,1,0.00
12801,24180,3.389958e+05,3.389958e+05,9.546745e+05,1.279698e+05,9.032086e+05,1,0,0,0,...,0,0,0,1.987976e+06,0,0,0,0,1,0.00


In [18]:
#Division del set de test y train:

division_x_train, division_x_test, division_y_train, division_y_test = train_test_split(x_train, y_train, test_size = 0.3,\
                                                                                       random_state = 123)

In [24]:
#Tuning de Hiper parametros con GridSearchCV

model = RandomizedSearchCV(CatBoostRegressor(),{
    'iterations' : [100,200,300,400,500],
    'learning_rate' : np.arange(0.005, 0.1, 0.005),
    'border_count' : [6,7,8,9,10],
    #'colsample_bytree' : np.arange(0.3,0.8,0.1),
    #'alpha' : [0.0001,0.001,0.01],
    'depth' : np.arange(5,9),
    #'l2_leaf_reg' : np.logspace(-20, 10),
    #'gamma' : np.arange(0,0.3,0.05),
    'random_strength' : np.arange(0.6,1,0.1)
    }, cv = 2, return_train_score = False, n_jobs = -1, verbose=10, n_iter=500)

model.fit(division_x_train, division_y_train)

Fitting 2 folds for each of 500 candidates, totalling 1000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    6.0s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:   15.6s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   22.2s
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   33.5s
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   48.9s
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done  61 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done  74 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  89 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 104 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 121 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 157 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 197 tasks      | elapsed:  

0:	learn: 0.4990071	total: 2.6ms	remaining: 258ms
1:	learn: 0.4983385	total: 4.91ms	remaining: 240ms
2:	learn: 0.4976188	total: 7.1ms	remaining: 230ms
3:	learn: 0.4969513	total: 9.65ms	remaining: 232ms
4:	learn: 0.4962445	total: 12.3ms	remaining: 234ms
5:	learn: 0.4955411	total: 14.6ms	remaining: 229ms
6:	learn: 0.4948515	total: 16.9ms	remaining: 224ms
7:	learn: 0.4941475	total: 19ms	remaining: 218ms
8:	learn: 0.4934944	total: 21.2ms	remaining: 214ms
9:	learn: 0.4927794	total: 23.4ms	remaining: 211ms
10:	learn: 0.4921547	total: 25.5ms	remaining: 206ms
11:	learn: 0.4914602	total: 27.8ms	remaining: 204ms
12:	learn: 0.4908190	total: 30ms	remaining: 201ms
13:	learn: 0.4902135	total: 32.1ms	remaining: 197ms
14:	learn: 0.4895383	total: 34.4ms	remaining: 195ms
15:	learn: 0.4889021	total: 36.6ms	remaining: 192ms
16:	learn: 0.4882558	total: 38.8ms	remaining: 189ms
17:	learn: 0.4876052	total: 41ms	remaining: 187ms
18:	learn: 0.4869926	total: 43.1ms	remaining: 184ms
19:	learn: 0.4863690	total: 45

RandomizedSearchCV(cv=2,
                   estimator=<catboost.core.CatBoostRegressor object at 0x000001FAC381E370>,
                   n_iter=500, n_jobs=-1,
                   param_distributions={'border_count': [6, 7, 8, 9, 10],
                                        'depth': array([5, 6, 7, 8]),
                                        'iterations': [100, 200, 300, 400, 500],
                                        'learning_rate': array([0.005, 0.01 , 0.015, 0.02 , 0.025, 0.03 , 0.035, 0.04 , 0.045,
       0.05 , 0.055, 0.06 , 0.065, 0.07 , 0.075, 0.08 , 0.085, 0.09 ,
       0.095]),
                                        'random_strength': array([0.6, 0.7, 0.8, 0.9])},
                   verbose=10)

In [20]:
model.score(division_x_test, division_y_test)

-2937.548753974669

In [21]:
model.best_params_

{'random_strength': 0.7,
 'learning_rate': 0.01,
 'l2_leaf_reg': 10000000000.0,
 'iterations': 500,
 'depth': 8,
 'border_count': 10}

In [22]:
log_loss(division_y_test, model.predict(division_x_test))

0.6921999862258471

In [23]:
accuracy_score(division_y_test, model.predict(division_x_test).round())

0.5228046289993192